In [13]:
import nltk
import numpy as np
import random
import lxml


import bs4 as bs
import urllib.request
import re
import ssl

In [52]:
#this piece of code filters the code files and cleans the non usefull files
import pathlib
import shutil
import os

def delete_subdirs(directory_path):
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        if os.path.isdir(file_path):
            shutil.rmtree(file_path)
        else:
            os.remove(file_path)

# sets the path of the directory where the python files are downloades
path = pathlib.Path('./python-repositories')
#sets the file where we are moving the python files
new_dir = pathlib.Path('./python-files')

#loops trough all the the files in the directory
for p in path.rglob('*.py'):
    new_path = new_dir / p.name  # set the new file path
    p.rename(new_path) 

#removes all the folders with the non python files
path = pathlib.Path('./python-repositories/')
delete_subdirs(path)

In [43]:
import nltk

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jonnasosauranga/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [80]:
import tokenize
import io

def clean_and_tokenize(code):
    # Remove comments and blank lines
    code = '\n'.join(line for line in code.split('\n') if not line.strip().startswith('#'))

    # Convert the code into a file-like object
    code_io = io.BytesIO(code.encode('utf-8'))

    # Tokenize the code
    tokens = tokenize.tokenize(code_io.readline)

    # Extract the token types and values
    token_types = []
    token_values = []
    for token in tokens:
        if token.type != tokenize.NEWLINE and token.type != tokenize.NL:
            token_types.append(tokenize.tok_name[token.type])
            token_values.append(token.string)

    return token_types, token_values


In [86]:
import os
import tokenize
import string

# Define the path to the directory containing the Python files
dir_path = './python-files'

# Create an empty list to store the code from all Python files
code_list = []

# Loop through each file in the directory and extract the code
for filename in os.listdir(dir_path):
    if filename.endswith('.py'):
        with open(os.path.join(dir_path, filename), 'r') as f:
            code_list.append(f.read())

# Concatenate the code from all files into a single string
code = '\n'.join(code_list)

# Count the number of lines of code
num_lines = code.count('\n')

# Tokenize the code by lowercasing everything and splitting on all whitespace and punctuation characters
tokens = []
for token in tokenize.tokenize(io.BytesIO(code.encode('utf-8')).readline):
    if token.type == tokenize.NAME:
        # Remove any leading or trailing spaces and punctuation from the token
        token_value = token.string.lower().strip(string.whitespace + string.punctuation)
        if token_value:
            tokens.append(token_value)

# Count the number of tokens
num_tokens = len(tokens)


In [89]:
# ngram dictionary to keep the possible ngrams as keys and next occuring words as list items
ngrams = {}

# words in a single ngram
words = 3

# Iterate over words tokens
for i in range(len(tokens)-words):
    # incrementaly find ngram word sequences  
    seq = ' '.join(tokens[i:i+words])

    # insert in dictionary as key
    if  seq not in ngrams.keys():
        # prepare empty list to insert next possible chars
        ngrams[seq] = []

    # push a next word
    ngrams[seq].append(tokens[i+words])

#print(ngrams)

In [90]:
def auto_complete(seed_sequence):
    # assuming very first ngram as a search sequence
    search_sequence = seed_sequence #' '.join(words_tokens[0:words])

    # init the suggestion output
    output = search_sequence

    next_word = ""

    # lenth of max words in obtained suggestion
    #while next_word != ".":
    for i in range(3):
        # break, if search sequence is not present in prepared ngram dictionary
        if search_sequence not in ngrams.keys():
            search_sequence = random.choice(list(ngrams.keys()))
            #print(search_sequence)
        
        # if ngram key is available, then find the list of next possible characters
        possible_words = ngrams[search_sequence]

        # Randomly select the next possbile character, most common will have more chances
        next_word = possible_words[random.randrange(len(possible_words))]

        # Update the suggestion output
        output += ' ' + next_word
        seq_words = nltk.word_tokenize(output)

        # update the search sequence now, excluding first char as we move forward
        search_sequence = ' '.join(seq_words[len(seq_words)-words:len(seq_words)])
        
    return output


In [93]:
seed_sequence = "test"
print(f'Search sequence: ', seed_sequence)
print(f'\nSuggestion: {auto_complete(seed_sequence)}')

Search sequence:  test

Suggestion: test other return raise


TO DO

In [25]:
import github 

# Replace <github_username> and <github_token> with your own username and personal access token
g = github.Github("<github_username>", "<github_token>")

# Replace <repo_owner> and <repo_name> with the owner and name of the repository you want to download
repo = g.get_repo("<repo_owner>/<repo_name>")

# Replace <download_path> with the path where you want to download the repository
download_path = "<download_path>"

# Download the repository to the specified path
repo.get_archive("zipball", download_path)

AttributeError: module 'github' has no attribute 'Github'